In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path

# Add src directory to Python path
sys.path.append(str(Path.cwd().parent / 'src'))
sys.path.append(str(Path.cwd().parent / 'config'))

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("📦 Libraries imported successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"🐼 Pandas version: {pd.__version__}")

## 🔌 Initialize Data Connector

Our data connector provides a unified interface for loading data from multiple sources.

In [ ]:
# Import our custom data connector
try:
    from data_connector import DataConnector, quick_load, load_demo_data
    print("✅ Data connector imported successfully")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("💡 Make sure you're running from the notebooks/ directory")

# Initialize the connector
connector = DataConnector()
print("🔧 Data connector initialized")

## 📊 Option 1: Demo Data (Always Available)

Let's start with demo data to ensure everything works properly.

In [ ]:
# Load demo data
demo_df = load_demo_data()

print(f"📈 Demo dataset shape: {demo_df.shape}")
print(f"📋 Columns: {list(demo_df.columns)}")
print(f"📅 Date range: {demo_df['Date'].min()} to {demo_df['Date'].max()}")

# Display first few rows
demo_df.head()

In [ ]:
# Quick data summary
print("📊 Demo Data Summary:")
print(f"Total rows: {len(demo_df)}")
print(f"Categories: {demo_df['Category'].unique()}")
print(f"Regions: {demo_df['Region'].unique()}")
print(f"Products: {demo_df['Product'].unique()}")
print(f"Value range: ${demo_df['Value'].min()} - ${demo_df['Value'].max()}")

demo_df.describe()

## 🗂️ Option 2: CSV File Loading

Load data from CSV files (local or URL).

In [ ]:
# Example: Load from CSV URL (using a public dataset)
csv_url = "https://raw.githubusercontent.com/plotly/datasets/master/tips.csv"

try:
    csv_df = connector.load_csv_url(csv_url)
    if csv_df is not None:
        print(f"✅ CSV data loaded: {csv_df.shape}")
        print(f"Columns: {list(csv_df.columns)}")
        display(csv_df.head())
    else:
        print("❌ Failed to load CSV data")
except Exception as e:
    print(f"⚠️ Error loading CSV: {e}")
    print("💡 This might be due to internet connectivity")

In [ ]:
# Example: Load from local CSV file
# First, let's save our demo data as a CSV for testing
csv_path = Path('../data/raw/demo_data.csv')
csv_path.parent.mkdir(parents=True, exist_ok=True)

demo_df.to_csv(csv_path, index=False)
print(f"💾 Saved demo data to: {csv_path}")

# Now load it back
loaded_csv = connector.load_csv_file(csv_path)
if loaded_csv is not None:
    print(f"✅ Local CSV loaded: {loaded_csv.shape}")
    print("✅ Data integrity check:", loaded_csv.equals(demo_df))

## 📋 Option 3: Google Sheets Integration

Connect to Google Sheets using the API. This requires authentication setup.

### 🔐 Authentication Methods:
1. **Service Account** (Recommended for Binder): Create a service account, download JSON key
2. **OAuth2** (Local development): Use Google account authentication
3. **Environment Variables**: Store credentials as environment variables

### 📝 Setup Steps:
1. Go to [Google Cloud Console](https://console.cloud.google.com/)
2. Create a new project or select existing
3. Enable Google Sheets API
4. Create credentials (Service Account)
5. Download the JSON key file
6. Share your Google Sheet with the service account email

In [ ]:
# Test Google Sheets configuration
try:
    from google_sheets_config import GoogleSheetsConfig
    
    # Initialize configuration
    sheets_config = GoogleSheetsConfig()
    
    # Try to setup credentials
    creds_setup = sheets_config.setup_credentials()
    
    if creds_setup:
        print("✅ Google Sheets credentials configured")
        
        # Get authenticated client
        client = sheets_config.get_client()
        
        if client:
            print("✅ Google Sheets client ready")
            print("🔗 You can now connect to Google Sheets!")
        else:
            print("❌ Failed to create Google Sheets client")
    else:
        print("❌ Google Sheets credentials not configured")
        print("💡 See authentication setup instructions above")
        
except ImportError:
    print("⚠️ Google Sheets dependencies not available")
    print("💡 Install with: pip install gspread google-auth google-auth-oauthlib")
except Exception as e:
    print(f"❌ Error setting up Google Sheets: {e}")

In [ ]:
# Example: Load from Google Sheets (replace with your sheet URL/ID)
# This is a public demo sheet - replace with your own
demo_sheet_url = "https://docs.google.com/spreadsheets/d/1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms/edit#gid=0"

try:
    sheets_df = connector.load_google_sheet(demo_sheet_url)
    
    if sheets_df is not None:
        print(f"✅ Google Sheets data loaded: {sheets_df.shape}")
        print(f"Columns: {list(sheets_df.columns)}")
        display(sheets_df.head())
    else:
        print("❌ Failed to load Google Sheets data")
        print("💡 This might be due to authentication or permissions")
        
except Exception as e:
    print(f"⚠️ Google Sheets loading error: {e}")
    print("💡 Using fallback data loading...")

## 🔄 Auto-Loading with Fallbacks

The `auto_load` function tries multiple methods and falls back to demo data if all fail.

In [ ]:
# Test auto-loading with different source types
test_sources = [
    "demo",  # Always works
    "https://raw.githubusercontent.com/plotly/datasets/master/iris.csv",  # CSV URL
    "../data/raw/demo_data.csv",  # Local CSV
    "nonexistent_file.csv",  # Will fail, fallback to demo
]

for source in test_sources:
    print(f"\n🧪 Testing source: {source}")
    df = connector.auto_load(source)
    print(f"   Result: {df.shape[0]} rows, {df.shape[1]} columns")
    print(f"   Columns: {list(df.columns)[:3]}..." if len(df.columns) > 3 else f"   Columns: {list(df.columns)}")

## 📊 Data Source Information

Track metadata about loaded data sources.

In [ ]:
# Display information about all loaded sources
source_info = connector.get_source_info()

print("📋 Data Source Summary:")
print("=" * 50)

for source_key, info in source_info.items():
    print(f"\n🔹 Source: {source_key}")
    print(f"   Type: {info.get('type', 'unknown')}")
    print(f"   Rows: {info.get('rows', 'unknown')}")
    print(f"   Columns: {info.get('columns', 'unknown')}")
    print(f"   Last Updated: {info.get('last_updated', 'unknown')}")
    
    # Additional type-specific info
    if info.get('type') == 'google_sheets':
        print(f"   Sheet Title: {info.get('sheet_title')}")
        print(f"   Worksheet: {info.get('worksheet_title')}")
    elif info.get('type') == 'csv_file':
        print(f"   File Size: {info.get('file_size', 0):,} bytes")
        print(f"   Encoding: {info.get('encoding')}")
    elif info.get('type') == 'csv_url':
        print(f"   URL: {info.get('url', '')[:50]}...")
        print(f"   Response Size: {info.get('response_size', 0):,} bytes")

if not source_info:
    print("No data sources loaded yet.")

## 🎯 Your Turn: Load Your Own Data

Now it's time to load your actual data! Follow these steps:

### For Google Sheets:
1. Replace the URL below with your Google Sheet URL
2. Make sure the sheet is shared with your service account email
3. Or make the sheet publicly viewable (anyone with link can view)

### For CSV files:
1. Upload your CSV to the `data/raw/` folder
2. Update the file path below

### For CSV URLs:
1. Use any publicly accessible CSV URL

In [ ]:
# 🔧 CUSTOMIZE THIS SECTION FOR YOUR DATA
# =====================================

# Option 1: Google Sheets URL
# Replace with your Google Sheets URL
YOUR_GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/YOUR_SHEET_ID/edit"

# Option 2: CSV file path
# Replace with your CSV file path
YOUR_CSV_PATH = "../data/raw/your_data.csv"

# Option 3: CSV URL
# Replace with your CSV URL
YOUR_CSV_URL = "https://example.com/your_data.csv"

# Choose your preferred data source
# Uncomment one of the following lines:

# data_source = YOUR_GOOGLE_SHEET_URL
# data_source = YOUR_CSV_PATH
# data_source = YOUR_CSV_URL
data_source = "demo"  # Default to demo data

print(f"🎯 Loading data from: {data_source}")

In [ ]:
# Load your data
your_data = connector.auto_load(data_source)

print(f"✅ Data loaded successfully!")
print(f"📊 Shape: {your_data.shape}")
print(f"📋 Columns: {list(your_data.columns)}")

# Display first few rows
display(your_data.head())

# Basic info about the dataset
print(f"\n📈 Data Types:")
print(your_data.dtypes)

print(f"\n🔍 Missing Values:")
missing_values = your_data.isnull().sum()
print(missing_values[missing_values > 0])

## 💾 Save Data for Further Analysis

Save the loaded data for use in other notebooks.

In [ ]:
# Save to processed data folder
output_path = Path('../data/processed/loaded_data.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

your_data.to_csv(output_path, index=False)
print(f"💾 Data saved to: {output_path}")

# Also save as Excel if you prefer
excel_path = Path('../data/processed/loaded_data.xlsx')
try:
    your_data.to_excel(excel_path, index=False)
    print(f"💾 Data also saved as Excel: {excel_path}")
except ImportError:
    print("💡 Install openpyxl to save Excel files: pip install openpyxl")

# Cache in memory for other notebooks
connector.save_to_cache(your_data, 'main_dataset')
print("💾 Data cached in memory as 'main_dataset'")

## 🎉 Summary

Congratulations! You've successfully set up data ingestion for your Insight Digger project.

### ✅ What You Accomplished:
- ✅ Set up the data connector system
- ✅ Loaded demo data for testing
- ✅ Tested CSV file loading (local and URL)
- ✅ Configured Google Sheets integration (if credentials available)
- ✅ Used auto-loading with fallback options
- ✅ Saved data for further analysis

### 🚀 Next Steps:
1. **Open `02_data_analysis.ipynb`** to start analyzing your data
2. **Configure Google Sheets API** if you haven't already
3. **Upload your own CSV files** to the `data/raw/` folder
4. **Share this notebook** with others via Binder

### 💡 Tips for Success:
- Keep your credentials secure and never commit them to Git
- Use the cache functionality to avoid reloading data
- Test with demo data first before connecting real sources
- Check the source info to understand your data pipeline

Happy data digging! 🕵️‍♂️